# Machine Learning - Homework 3

**Goal** : Translate the python program to TensorFlow using the elab Cloud Jupyter Notebook

**Data** : Data is taken from sklearn's make_moon dataset. There are two features and and the target is a categorical variable (0/1). The aim is to devise an algorithm that correctly classifies the datapoints. Here is the link to the data set: (https://github.com/ml6973/Course/blob/master/code/data/intro_to_ann.csv)

**Aproach**: Build a TensorFlow neural networks Program from first principles

In [8]:
import csv
import pandas as pd
import tensorflow as tf
import numpy as np
from numpy import genfromtxt
import matplotlib
import matplotlib.pyplot as plt
import math as math
import random
import string

###############  Global variables  ###############
NO_LABELS = 2  # The number of labels.
BATCH_SIZE = 100  # The number of training examples to use per training step.
# CSV file variable
file_name = 'data/intro_to_ann.csv'

In [9]:
# Function to read the csv file
def read_data_file(file_name):
    df = pd.read_csv(file_name)
    feature1 = df['Feature1']
    feature2 = df['Feature2']
    label = df['Target']
    feats=[feature1, feature2]
    feature1 = pd.concat(feats, axis=1)
    print(feature1.shape)
    return feature1, label


In [10]:
# Weights initializer
def initializer(shape, init_method='zeros'):
    if init_method == 'zeros':
        return tf.Variable(tf.zeros(shape, dtype=tf.float32))
    elif init_method == 'uniform':
        return tf.Variable(tf.random_normal(shape, stddev=0.01, dtype=tf.float32))

In [11]:
# Prediction for the  next layers etc
# Referred to internet sources for this prediction algo.
def prediction_mapper(x, w_hidden, b_hidden, w_out, b_out):
    hidden = tf.nn.tanh(tf.matmul(tf.cast(x, tf.float32), w_hidden) + b_hidden)
    y = tf.nn.softmax(tf.matmul(tf.cast(hidden, tf.float32), w_out) + b_out)
    # Debugging  of the prediction
    print("w_hidden b_hidden w_out b_out")
    print ("w_hidden", w_hidden.eval())
    print ("b_hidden", b_hidden.eval())
    print ("w_out", w_out.eval())
    print ("b_out", b_out.eval())
    predicted_value = tf.argmax(y, 1)
    return predicted_value.eval()

In [12]:
# Helper function in place of original xrange() in python2
def xrange(x):
    return iter(range(x))

In [13]:
#################### Training data section ######################
# Extract the values from the csv file into the it into numpy arrays.
train_data, train_labels = read_data_file(file_name)
print("Training Data")
print(train_data)
print("Training Labels")
print(train_labels)
# Get the shape of the training data.
train_data = np.matrix(train_data)
print("Training Matrix")
print(train_data)
# Get the size and features in the training data
train_size, num_features = train_data.shape

#################### Test data section ######################
test_data, test_labels = read_data_file(file_name)  # Using the  same file.. for now

# Test data will be placed as a constant


num_epochs = 1000
num_hidden = 2

# Create placeholders to be used in the tensorflow execution Run
x = tf.placeholder(tf.float32, shape=[None, num_features])
y_ = tf.placeholder(tf.float32, shape=[None, NO_LABELS])

(500, 2)
Training Data
    Feature1  Feature2
0   2.067788  0.258133
1   0.993994 -0.609145
2  -0.690315  0.749921
3   1.023582  0.529003
4   0.700747 -0.496724
5   0.955063  0.371061
6  -0.051023  0.009787
7   2.111669  0.291467
8   1.173019 -0.101473
9  -0.577947  1.277303
10  0.909598  0.036309
11  2.077734  0.629148
12 -0.962291  0.827234
13  0.947004 -0.290186
14  0.383210 -0.309918
15 -0.150288  0.317294
16  0.080361  0.502094
17  1.200125 -0.527865
18 -0.922387  0.465575
19  0.512395 -0.033492
20  0.660177  0.354199
21  0.919879 -0.593772
22  0.185474 -0.214732
23 -0.963168  0.294004
24  1.426700 -0.099509
25  0.976458 -0.449374
26  1.754714 -0.261643
27  1.168187 -0.672741
28  0.277747 -0.607361
29  0.672725  0.666334
30  0.002033  0.376968
31  0.314327 -0.243226
32  0.931967  0.173112
33 -0.409800  0.597132
34 -0.476941  1.019582
35  0.838883 -0.071573
36  1.033285 -0.525250
37  0.326116 -0.577021
38  0.859473  1.340803
39  1.551222 -0.528586
40  0.101201  0.540632
41 -0.09049

In [14]:
# Initialize the hidden weights and biases.
w_hidden = initializer([num_features, num_hidden],'zeros')
print("w_hidden", w_hidden)
b_hidden = initializer([1, num_hidden], 'zeros')
print("b_hidden", b_hidden)


################### Design the Hidden Layer & outputs ###################
#   Hidden Layer
#   Output Layers
#   Optimization process
#   Validation of results

hidden = tf.nn.tanh(tf.matmul(x, w_hidden) + b_hidden)

# Initialize the output weights and biases.
w_out = initializer([num_hidden, NO_LABELS], 'zeros')
b_out = initializer([1, NO_LABELS], 'zeros')

# The output layer.
y = tf.nn.softmax(tf.matmul(hidden, w_out) + b_out)

# Optimization.
cross_entropy = -tf.reduce_sum(y_ * tf.log(y))
train_step = tf.train.GradientDescentOptimizer(0.01).minimize(cross_entropy)
print("train_step -- ", train_step)

# Validation of results
correct_prediction = tf.equal(tf.argmax(y, 1), tf.argmax(y_, 1))

accuracy = tf.reduce_mean(tf.cast(correct_prediction, "float"))
model = tf.initialize_all_variables()

w_hidden <tensorflow.python.ops.variables.Variable object at 0x7fa3f68f0780>
b_hidden <tensorflow.python.ops.variables.Variable object at 0x7fa3f68bdd30>
train_step --  name: "GradientDescent_1"
op: "NoOp"
input: "^GradientDescent_1/update_Variable_4/ApplyGradientDescent"
input: "^GradientDescent_1/update_Variable_5/ApplyGradientDescent"
input: "^GradientDescent_1/update_Variable_6/ApplyGradientDescent"
input: "^GradientDescent_1/update_Variable_7/ApplyGradientDescent"



In [15]:
####################  Tensor exection section  ###################
# Create a tensorflow session
with tf.Session() as sess:
    # Run all the initializers to prepare the trainable parameters.
    sess.run(model)
    coord = tf.train.Coordinator()
    threads = tf.train.start_queue_runners(coord=coord)

    # step through and train the system extract the label, data set and epoch
    for step in xrange(num_epochs * train_size // BATCH_SIZE):
        offset = (step * BATCH_SIZE) % train_size
        batch_data = train_data[offset:(offset + BATCH_SIZE), :]
        batch_labels = train_labels[offset:(offset + BATCH_SIZE)]
        print("Batch_data",batch_data)        
        print("Batch_Labels", batch_labels)
        train_step.run(feed_dict={x: batch_data, y_: batch_labels})
        print(accuracy.eval(feed_dict={x: test_data[:1], y_: test_labels}))

        # Train the system
        train = pd.read_csv("data/intro_to_ann.csv")
        X, y = np.array(train.ix[:, 0:2]), np.array(train.ix[:, 2])
        x_min, x_max = X[:, 0].min() - .5, X[:, 0].max() + .5
        y_min, y_max = X[:, 1].min() - .5, X[:, 1].max() + .5
        stepsize = 0.01
        
        # Generate a grid of points with distance h between them
        x_points, y_points = np.meshgrid(np.arange(x_min, x_max, stepsize), np.arange(y_min, y_max, stepsize))
    
    print("Optimization Completed!")
    # Predict and map the grid
    Pred = prediction_mapper(np.c_[x_points.ravel(), y_points.ravel()], w_hidden, b_hidden, w_out, b_out)
    Pred = Pred.reshape(x_points.shape)

    # Plot the contour and training examples
    plt.contourf(x_points, y_points, Pred, cmap=plt.cm.Spectral)

    plt.scatter(X[:, 0], X[:, 1], s=40, c=2, cmap=plt.cm.BuGn)
    plt.show()

Batch_data [[  2.06778839e+00   2.58133225e-01]
 [  9.93994008e-01  -6.09144512e-01]
 [ -6.90315436e-01   7.49920622e-01]
 [  1.02358238e+00   5.29003080e-01]
 [  7.00747058e-01  -4.96724018e-01]
 [  9.55062941e-01   3.71061016e-01]
 [ -5.10234660e-02   9.78688300e-03]
 [  2.11166892e+00   2.91466670e-01]
 [  1.17301939e+00  -1.01473076e-01]
 [ -5.77947070e-01   1.27730309e+00]
 [  9.09597624e-01   3.63086720e-02]
 [  2.07773424e+00   6.29147857e-01]
 [ -9.62290964e-01   8.27233623e-01]
 [  9.47003591e-01  -2.90186106e-01]
 [  3.83209976e-01  -3.09917567e-01]
 [ -1.50287574e-01   3.17294355e-01]
 [  8.03611230e-02   5.02094351e-01]
 [  1.20012519e+00  -5.27865179e-01]
 [ -9.22386507e-01   4.65574823e-01]
 [  5.12394856e-01  -3.34921280e-02]
 [  6.60176708e-01   3.54198518e-01]
 [  9.19878779e-01  -5.93771970e-01]
 [  1.85474117e-01  -2.14731663e-01]
 [ -9.63168026e-01   2.94003942e-01]
 [  1.42669965e+00  -9.95090790e-02]
 [  9.76457825e-01  -4.49373622e-01]
 [  1.75471374e+00  -2.6164

ValueError: Cannot feed value of shape (100,) for Tensor 'Placeholder_3:0', which has shape '(?, 2)'